# **Analysis of Airbnb data on multiple locations spread across Spain**


In this mini-project, I use the [CRISP-DM](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining) process to answer several business questions about Airbnb locations and reservations across Spain using their publicly-available data. Get to know the main insights by reading [my post on Medium](https://medium.com/@carlosuziel.pm/can-we-guess-the-best-location-in-spain-to-invest-in-real-state-9a7c74304456).


## **Section 1: Business Understanding**

---


We will take the role of a private investor that has decided to purchase a property in Spain for renting it out through Airbnb. After careful examination, we have selected 9 possible Spanish cities where it would be interesting to make such a purchase. Naturally, we want to maximize our return on investment (ROI), for which we need to understand the competition in each city as well as the main price drivers for each location.

After having a brief look at the available data, we have selected a few questions that will aid us in making our investment decisions:

1. _**What is the average price of each location type per neighbourhood? What are the most expensive neighbourhoods on average?**_
2. _**What is the average host acceptance rate per location type and neighborhood? In which neighbourhoods is it the lowest?**_
3. _**How is the competition in each neighbourhood? What number and proportion of listings belong to hosts owning different numbers of locations? In which neighbourhoods is the concentration lower?**_
4. _**What is the expected average profit per room type and neighborhood when looking at the reservations for the next 6 months? What is the neighbourhood expected to be the most profitable in that period?**_
5. _**What listings' factors affect the expected profit for the next 6 months? Can we use them to forecast the expected profit over that period?**_

We will be comparing the answers to those questions among the different Spanish regions of **Madrid**, **Barcelona**, **Girona**, **Valencia**, **Mallorca**, **Menorca**, **Sevilla**, **Málaga** and **Euskadi**. Hopefully, this will help us in making a more informed investment decision.


## **Section 2: Data Understanding**

---


In this section we will be taking a brief look at the data we obtained from the [Airbnb portal](http://insideairbnb.com/get-the-data/). We are going to be examining multilple Spanish regions, but since all of them share the same data structure, here we will only focus on Madrid.


### Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
import logging
import json
import pandas as pd

from pathlib import Path
from IPython import display as ICD

In [ ]:
src_path: str = "../src"
sys.path.append(src_path)
logging.getLogger().setLevel(logging.WARN)

In [ ]:
from data_wrangling import (
    airbnb_avg_price,
    airbnb_avg_accept_rate,
    airbnb_hood_hosts,
    airbnb_avg_profit,
    airbnb_predict_profit,
)

In [ ]:
random_seed: int = 8080
airbnb_root: Path = Path("../data/airbnb")
madrid_root: Path = airbnb_root.joinpath("madrid")
results_root: Path = Path("../data/results")

### Gather data


In [ ]:
listings_schema_df = pd.read_csv(
    madrid_root.parent.joinpath("listings_schema.csv")
).set_index("Field")
listings_df = pd.read_csv(madrid_root.joinpath("listings.csv.bz2"))
calendar_df = pd.read_csv(madrid_root.joinpath("calendar.csv.bz2"))

#### Listings


The following are the fields of the listings dataframe that **have** descriptions:


In [ ]:
listings_with_desc = listings_schema_df[["Description"]].dropna()
listings_with_desc

The following are the fields of the listings dataframe that **don't have** descriptions:


In [ ]:
listings_without_desc = listings_schema_df[["Description"]][
    listings_schema_df["Description"].isna()
]
listings_without_desc.index.tolist()

Most fields without descriptions are self-explanatory. To answer our questions, we will mostly be using the following fields: `id`, `price`, `neighbourhood_cleansed` and `room_type`. Additional fields will be used in concrete cases, such as the final data modeling step to answer question 5.


### Calendar


In [ ]:
calendar_df.head()

The calendar data is quite simple. Each row shows the availability of a listing at a certain date, which includes an `adjusted_price` that we assume is the final price to pay by the person making the reservation, should it not be cancelled beforehand.


## **Section 3: Data Preparation**

---


All data preparation steps are included in functions located in `src/data_wrangling.py` following good software engineering principles such as reproducibility and encapsulation. Each function is designed to separately prepare the data necessary to answer each question. They are thus complete and independent of each other. In this section we will briefly see the data processing steps that were necessary to answer each question.


### **Question 1: _What is the average price of each location type per neighbourhood? What are the most expensive neighbourhoods on average?_**


In [ ]:
most_expensive_hoods = {}
for path in airbnb_root.glob("**/listings.csv.bz2"):
    region_name = path.parent.name
    df, most_expensive_hoods[region_name.title()] = airbnb_avg_price(path)
    df.to_csv(results_root.joinpath(f"question_1_{region_name}.csv"))

most_expensive_hoods_df = pd.DataFrame(most_expensive_hoods).transpose().sort_index()
most_expensive_hoods_df.to_csv(results_root.joinpath("question_1_summary.csv"))

The function `airbnb_avg_price` provides two outputs, with the first and second answering the first and second parts of the question, respectively.

Data processing steps necessary to answer this question include:

1. Load listings data.
2. Dropping NaN values in the fields `price`, `room_type` and `neighbourhood_cleansed`. I decided to drop them because imputing these would make no sense.
3. Ensure that only string values are present in the fields `price`, `room_type` and `neighbourhood_cleansed`. Non-string values are dropped without consideration, as they are considered to be human-made errors. Besides, this was an issue only affecting a few regions.
4. Extract numeric value from `price` string field, which originally includes currency symbol. The new field is called `price_num`.
5. Group data by `neighbourhood_cleansed` and `room_type` and average `price_num`.
6. Finally, we sort results to obtain the most expensive neighbourhoods on average per room type.


### **Question 2: _What is the average host acceptance rate per location type and neighborhood? In which neighbourhoods is it the lowest?_**

This can give us an idea of the negotiating power of the hosts or the desirability of guests.


In [ ]:
lowest_accept_rate_hoods = {}
for path in airbnb_root.glob("**/listings.csv.bz2"):
    region_name = path.parent.name
    df, lowest_accept_rate_hoods[region_name.title()] = airbnb_avg_accept_rate(path)
    df.to_csv(results_root.joinpath(f"question_2_{region_name}.csv"))

lowest_accept_rate_hoods_df = (
    pd.DataFrame(lowest_accept_rate_hoods).transpose().sort_index()
)
lowest_accept_rate_hoods_df.to_csv(results_root.joinpath("question_2_summary.csv"))

The function `airbnb_avg_accept_rate` provides two outputs, with the first and second answering the first and second parts of the question, respectively.

Data processing steps necessary to answer this question include:

1. Load listings data.
2. Dropping NaN values in the fields `host_acceptance_rate`, `room_type` and `neighbourhood_cleansed`. I decided to drop them because imputing these would make no sense.
3. Ensure that only string values are present in the fields `host_acceptance_rate`, `room_type` and `neighbourhood_cleansed`. Non-string values are dropped without consideration, as they are considered to be human-made errors. Besides, this was an issue only affecting a few regions.
4. Extract numeric value from `host_acceptance_rate` string field, which originally includes percentage symbol. The new field is called `host_acceptance_rate_num`.
5. Group data by `neighbourhood_cleansed` and `room_type` and average `host_acceptance_rate_num`.
6. Finally, we sort results to obtain the neighbourhoods with the lowest acceptance rate on average per room type.


### **Question 3. _What number and proportion of listings per neighbourhood belong to hosts owning different numbers of locations? In which neighbourhoods is the concentration lower?_**


In [ ]:
least_dense_hoods = {}
for path in airbnb_root.glob("**/listings.csv.bz2"):
    region_name = path.parent.name
    df, least_dense_hoods[region_name.title()] = airbnb_hood_hosts(path)
    df.to_csv(results_root.joinpath(f"question_3_{region_name}.csv"))

least_dense_hoods_df = pd.DataFrame(least_dense_hoods).transpose().sort_index()
least_dense_hoods_df.to_csv(results_root.joinpath("question_3_summary.csv"))

The function `airbnb_hood_hosts` provides two outputs, with the first and second answering the first and second parts of the question, respectively.

Data processing steps necessary to answer this question include:

1. Load listings data.
2. Dropping NaN values in the fields `id`, `host_id` and `neighbourhood_cleansed`. I decided to drop them because imputing these would make no sense.
3. Ensure that only string values are present in the field `neighbourhood_cleansed`. Non-string values are dropped without consideration, as they are considered to be human-made errors. Besides, this was an issue only affecting a few regions.
4. Group data by `host_id` and `neighbourhood_cleansed` to get the number of listings per host in each neighbourhood.
5. Group hosts into clusters (binning) depending on how many listings they own. We considered four clusters: `1`, `2_to_5`, `6_to_20` and `21_to_many`. Then, we count the number of hosts in each cluster for each neighbourhood.
6. Finally, we sort results to obtain the neighbourhoods with the lowest concentration of hosts in each host group.


### **Question 4: _What is the expected average profit per room type and neighborhood when looking at the reservations for the next 6 months? What is the neighbourhood expected to be the most profitable in that period?_**

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


In [ ]:
n_weeks = 6 * 4

In [ ]:
most_profitable_hoods = {}
for listings_path, calendar_path in zip(
    sorted(airbnb_root.glob("**/listings.csv.bz2")),
    sorted(airbnb_root.glob("**/calendar.csv.bz2")),
):
    region_name = listings_path.parent.name
    df, most_profitable_hoods[region_name.title()] = airbnb_avg_profit(
        listings_path, calendar_path, n_weeks=n_weeks
    )
    df.to_csv(results_root.joinpath(f"question_4_{region_name}.csv"))

most_profitable_hoods_df = pd.DataFrame(most_profitable_hoods).transpose().sort_index()
most_profitable_hoods_df.to_csv(results_root.joinpath("question_4_summary.csv"))

The function `airbnb_avg_profit` provides two outputs, with the first and second answering the first and second parts of the question, respectively.

Data processing steps necessary to answer this question include:

1. Load listings and calendar data.
2. **[Calendar]** Transform date column to datetime and filter to only include the number of weeks are are interested in (defined by `n_weeks`).
3. **[Calendar]** Drop non-numeric values in `listing_id` and numeric values in `adjusted_price`. I assume they are human errors.
4. **[Calendar]** Extract numeric value from `adjusted_price_num` string field, which originally includes currency symbol. The new field is called `adjusted_price_num`.
5. **[Listings]** Dropping NaN values in the fields `room_type`, `neighbourhood_cleansed` and `id`. I decided to drop them because imputing these would make no sense.
6. **[Listings]** Ensure that only string values are present in the fields `room_type` and `neighbourhood_cleansed`. Non-string values are dropped without consideration, as they are considered to be human-made errors. Besides, this was an issue only affecting a few regions. Also ensure the field `id` only contains numeric values.
7. Filter calendar rows where `available` equals "f". Then group by `listing_id` and sum `adjusted_price_num`. Finally, join table with listings using `id` as index.
8. Average expected profits in the observed period per neighbourhood and room type.
9. Finally, we sort results to obtain the most profitable neighbourhoods in the observed period per room type.


### **Question 5. _What listings' factors affect the total profit in the next 6 months? Can the total profit be predicted?_**

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


In [ ]:
profit_predictions = {}
for listings_path, calendar_path in zip(
    sorted(airbnb_root.glob("**/listings.csv.bz2")),
    sorted(airbnb_root.glob("**/calendar.csv.bz2")),
):
    region_name = listings_path.parent.name
    num_features, cat_features, mean_r2_score = airbnb_predict_profit(
        listings_path,
        calendar_path,
        feature_th=0,
        n_weeks=n_weeks,
        random_seed=random_seed,
    )

    results_root.joinpath(f"question_5_num_features_{region_name}.txt").write_text(
        "\n".join(num_features)
    )
    results_root.joinpath(f"question_5_cat_features_{region_name}.txt").write_text(
        "\n".join(cat_features)
    )

    profit_predictions[region_name.title()] = {
        "num_features": len(num_features),
        "cat_features": len(cat_features),
        "mean_r2_score": mean_r2_score,
    }

profit_predictions_df = pd.DataFrame(profit_predictions).transpose().sort_index()
profit_predictions_df.to_csv(results_root.joinpath("question_5_summary.csv"))

The function `airbnb_predict_profit` provides two outputs, with the first and second answering the first and second parts of the question, respectively.

Data processing steps necessary to answer this question include:

1. Load listings and calendar data.
2. **[Calendar]** Transform date column to datetime and filter to only include the number of weeks are are interested in (defined by `n_weeks`).
3. **[Calendar]** Drop non-numeric values in `listing_id` and numeric values in `adjusted_price`. I assume they are human errors.
4. **[Calendar]** Extract numeric value from `adjusted_price_num` string field, which originally includes currency symbol. The new field is called `adjusted_price_num`.
5. **[Listings]** Dropping NaN values in the fields `room_type`, `neighbourhood_cleansed` and `id`. I decided to drop them because imputing these would make no sense.
6. **[Listings]** Ensure that only string values are present in the fields `price`, `room_type` and `neighbourhood_cleansed`. Non-string values are dropped without consideration, as they are considered to be human-made errors. Besides, this was an issue only affecting a few regions. Also ensure the field `id` only contains numeric values.
7. **[Listings]** Extract numeric value from `price` string field, which originally includes currency symbol. The new field is called `price_num`.
8. **[Listings]** Remove possible confounding variables for our predictions. I removed all fields including the keywords "availability" and/or "calculated".
9. Filter calendar rows where `available` equals "f". Then group by `listing_id` and sum `adjusted_price_num`. Finally, join table with listings using `id` as index.
10. Average expected profits in the observed period per neighbourhood and room type. New field is called `total_profit`.
11. Calculate spearman correlation coefficient between `total_profit` and any other field describing a listing, be it a numeric field or a categorical field that has previously been encoded using dummy fields.
12. Choose the numeric features (`num_features`) and categorical features (`cat_features`) to be used for data modelling, based on the function argument `feature_th`, which determines the correlation cutoff used to select said features.
13. Next steps will be detailed in the data modelling part.


## **Section 4: Data Modelling**

---


### **Question 5. _What listings' factors affect the total profit in the next 6 months? Can the total profit be predicted?_**

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


To answer the second part of question 5, we must model our data to make predictions on total profits. Continuing after the data preparation detailed in the previous section, here are the additional steps necessary to train our predictive model included in `airbnb_predict_profit`:

1. Define the input matrix by subsetting the listings dataframe with the chosen numeric and categorical features.
2. Define the tartgets variable to be used, in this case `total_profit`.
3. Evaluate a [LightGBM](https://lightgbm.readthedocs.io) regression model using 10-fold cross validation. LightGBM is a gradient boosting ensemble machine learning algorithm that is known for achieving great predictive results while being very computationally efficient. In each fold, we train a [LGBMRegressor](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html) model and measure performance using [R2 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html). This roughly translates to the percentage of the variability in the target variable the model is able to explain.

## **Section 5: Evaluation of results**

---


Let's take a look at the results we obtained when trying to answer each of our business questions.

In [ ]:
def print_results(q_idx: int):
    """
    Load and print results answering business questions.

    Args:
        q_idx: Question number whose results to load.
    """
    for file_path in sorted(results_root.glob(f"question_{q_idx}_*.csv")):
        if "summary" in file_path.stem:
            continue
        print(f"Question {q_idx} - {file_path.stem.split('_')[-1].title()}:")
        ICD.display(pd.read_csv(file_path, index_col=0).head(9))
        print("\n")

    print(f"Question {q_idx} - Summary output for all Spanish regions:")
    ICD.display(
        pd.read_csv(
            results_root.joinpath(f"question_{q_idx}_summary.csv"), index_col=0
        ).head(9)
    )

### **Question 1: _What is the average price of each location type per neighbourhood? What are the most expensive neighbourhoods on average?_**


There are four types of listings in Airbnb. We can either rent an entire home or apartment, a hotel room, a private room or a shared room. Depending on which modality we are interesting in, now we know which neighbourhoods in each Spanish region are the most expensive on average:

In [ ]:
print_results(1)

### **Question 2: _What is the average host acceptance rate per location type and neighborhood? In which neighbourhoods is it the lowest?_**


Each listing has an acceptance rate value. We can hypothesize that the lower the acceptance rate on average, the higher the demand for rented properties there is in that area. This means hosts have a stronger negotiating power and can discard guests at will. This can be used as an indication of the most attractive areas that need to be served:

In [ ]:
print_results(2)

### **Question 3. _What number and proportion of listings per neighbourhood belong to hosts owning different numbers of locations? In which neighbourhoods is the concentration lower?_**


We can easily see which listings belong to which hosts. In some areas, there are a few hosts owning many of the listed places. These are probably real state agencies or wealthy people that control many Airbnb properties at the same time. If we are going to invest in a certain area, we should make sure that the bigger players are not going to pose a thread (e.g. through better means to attract guests). Therefore, we look at the neighbourhoods with the least concentration of competitors:

In [ ]:
print_results(3)

### **Question 4: _What is the expected average profit per room type and neighborhood when looking at the reservations for the next 6 months? What is the neighbourhood expected to be the most profitable in that period?_**

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


To answer this question, we need to look at the calendar information that we also obtained from Airbnb. There we can take a look at the dates that have already been reserved. If we assume that these are a good indication of general trend over time, then we can use this information to calculate the expected profit over a certain period. These would be the neighbourhoods for each Spanish region with the highest profits in the next 6 months, starting in September 2022:

In [ ]:
print_results(4)

### **Question 5. _What listings' factors affect the total profit in the next 6 months? Can the total profit be predicted?_**

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


Finally, we would like to know if the characteristics of a listing, such as the number of people it accommodates or the neighbourhood they are in, can be used to predict the total profit over a period of 6 months. Here we use a machine learning algorithm, LightGBM, to model our data and make predictions. Here is the final R2 score for each Spanish region, which roughly translates to the variance in the output captured by the model:

In [ ]:
print_results(5)